---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

def blight_model():
    
    # Your code here
    train_df = pd.read_csv('train.csv', encoding = "ISO-8859-1") 
    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    
    train_df = train_df[~train_df['compliance'].isnull()]
    
    addresses =  pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    addresses = addresses.set_index('address')
    latlons = latlons.set_index('address')
    addresses = addresses.join(latlons, how='left')
    addresses = addresses.set_index('ticket_id')
     
    train_df = train_df.set_index('ticket_id').join(addresses, how='left')
    test_df = test_df.set_index('ticket_id').join(addresses,  how='left')
    
    drop_cols_train = [ 'balance_due', 'collection_status', 'compliance_detail', 'payment_amount', 
                        'payment_date', 'payment_status' ]

    drop_cols = ['agency_name', 'inspector_name', 'violator_name',  'violation_street_name', 'violation_street_number',
                          'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state',
                          'zip_code', 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code',
                          'violation_description', 'disposition', 'grafitti_status']

    train_df.drop(drop_cols_train, axis=1, inplace=True)
    train_df.drop(drop_cols, axis=1, inplace=True)
    test_df.drop(drop_cols, axis=1, inplace=True)
    
    train_df.lat.fillna(method='pad', inplace=True)
    train_df.lon.fillna(method='pad', inplace=True)

    test_df.lat.fillna(method='pad', inplace=True)
    test_df.lon.fillna(method='pad', inplace=True)
    
    X_train = train_df.loc[:, train_df.columns != 'compliance']
    y_train = train_df[['compliance']]
    X_test = test_df
    
    clf = GradientBoostingClassifier(random_state=0)
    clf.fit(X_train, y_train)

    test_prob = clf.predict_proba(X_test)[:,1]

    test_df['compliance'] = test_prob
    
    return test_df['compliance']
    # Your answer here

In [ ]:
# blight_model()

In [2]:
train_df = pd.read_csv('train.csv', encoding = "ISO-8859-1") 
print(train_df.shape)
train_df.head(2)

(250306, 34)


/Users/cindyhui/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0


In [3]:
test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
print(test_df.shape)
test_df.head(2)

(61001, 27)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN


In [4]:
addresses =  pd.read_csv('addresses.csv')
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [5]:
latlons = pd.read_csv('latlons.csv')
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [6]:
train_df['agency_name'].value_counts()

Buildings, Safety Engineering & Env Department    157784
Department of Public Works                         74717
Health Department                                   8903
Detroit Police Department                           8900
Neighborhood City Halls                                2
Name: agency_name, dtype: int64

In [7]:
len(train_df[train_df['violation_zip_code'].isnull()])

250306

In [8]:
train_df['violation_code'].value_counts()

9-1-36(a)             99091
9-1-81(a)             43471
22-2-88               28720
9-1-104               22536
22-2-88(b)             7238
                      ...  
9-1-332                   1
61-84.0100/32.0031        1
61-4-38                   1
9-1-377                   1
61-105.0100/32.066        1
Name: violation_code, Length: 235, dtype: int64

In [9]:
train_df['compliance'].value_counts()
len(train_df[train_df['compliance'].isnull()])

90426

In [10]:
train_df = train_df[~train_df['compliance'].isnull()]

In [12]:
addresses = addresses.set_index('address')
latlons = latlons.set_index('address')
addresses = addresses.join(latlons, how='left')
addresses = addresses.set_index('ticket_id')
addresses.head()

,lat,lon
ticket_id,,
328722,42.406935,-82.995599
350971,42.411288,-82.993674
344821,42.406402,-82.957525
24928,42.325689,-83.064330
71887,42.325689,-83.064330


In [13]:
train_df = train_df.set_index('ticket_id').join(addresses, how='left')
test_df = test_df.set_index('ticket_id').join(addresses,  how='left')
print(train_df.shape)
print(test_df.shape)

(159880, 35)
(61001, 28)


In [14]:
drop_col_train = [ 'balance_due', 'collection_status', 'compliance_detail', 'payment_amount', 
                        'payment_date', 'payment_status' ]

drop_col = ['agency_name', 'inspector_name', 'violator_name',  'violation_street_name', 'violation_street_number',
                      'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state',
                      'zip_code', 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code',
                      'violation_description', 'disposition', 'grafitti_status']
   
train_df.drop(drop_col_train, axis=1, inplace=True)
train_df.drop(drop_col, axis=1, inplace=True)
test_df.drop(drop_col, axis=1, inplace=True)

print(train_df.shape)
print(test_df.shape)

(159880, 10)
(61001, 9)


In [15]:
train_df.lat.fillna(method='pad', inplace=True)
train_df.lon.fillna(method='pad', inplace=True)

test_df.lat.fillna(method='pad', inplace=True)
test_df.lon.fillna(method='pad', inplace=True)

X_train = train_df.loc[:, train_df.columns != 'compliance']
y_train = train_df[['compliance']]
X_test = test_df

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(159880, 9)
(159880, 1)
(61001, 9)


In [17]:
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)

test_prob = clf.predict_proba(X_test)[:,1]

test_df['compliance'] = test_prob

print(test_df['compliance'])


/Users/cindyhui/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


ticket_id
284932    0.055029
285362    0.020772
285361    0.057549
285338    0.081691
285346    0.089565
            ...   
376496    0.022862
376497    0.022862
376499    0.058873
376500    0.058873
369851    0.266408
Name: compliance, Length: 61001, dtype: float64
